# Item-based Models

In this file we are going to proceed with the building in the models. Machine learning models to build recommender systems. The models that are going to be built are Collaborative filtering using Item-based rating prediction (ItemKNN) and Item-based classification (ItemKNN).

Collaborative filtering is a technique used in recommendation systems to predict or classify items based on the preferences or behavior of similar users or items. Item-based Collaborative Filtering (CF) focuses on the similarity between items rather than users. There are two main approaches within item-based CF: rating prediction and classification.

In [81]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

In [82]:
# We import the necessary libraries
import pandas as pd
import numpy as np
import os

In [83]:
# We print the directory where the file is located
print(os.getcwd())

c:\Users\Jaume\Documents\MDDB\SDM\SDfM---Jaume-and-Stijn


In [84]:
# We set the directory to the cleaned folder
os.listdir(os.path.join('.', 'cleaned'))

['final_sample5_parquet', 'movielens_parquet', 'netflix_parquet']

In [85]:
# We read the final_sample file and store it in a dataframe
df = pd.read_parquet('cleaned/final_sample5_parquet')

In [86]:
# We print shape of the dataframe
df.shape

(4665, 26)

In [87]:
# We print the first 5 rows of the dataframe
df.head()

,movieId,title,year,userId,rating,date,Drama,Action,Sci-Fi,Comedy,...,Adventure,Fantasy,IMAX,Animation,Musical,Horror,Film-Noir,Western,Mystery,(no genres listed)
0,45635,"Notorious Bettie Page, The",2005,414,3.0,2008-07-15,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,45635,"Notorious Bettie Page, The",2005,474,3.0,2006-12-08,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1373,Star Trek V: The Final Frontier,1989,19,1.0,2000-08-08,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1373,Star Trek V: The Final Frontier,1989,42,4.0,2001-07-27,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1373,Star Trek V: The Final Frontier,1989,51,5.0,2009-01-02,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
# We print the columns of the dataframe
df.columns

Index(['movieId', 'title', 'year', 'userId', 'rating', 'date', 'Drama',
       'Action', 'Sci-Fi', 'Comedy', 'Crime', 'Thriller', 'Romance', 'War',
       'Documentary', 'Children', 'Adventure', 'Fantasy', 'IMAX', 'Animation',
       'Musical', 'Horror', 'Film-Noir', 'Western', 'Mystery',
       '(no genres listed)'],
      dtype='object')

In [89]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())


532
487


# Item-based Rating Prediction (ItemKNN)

### Step 1: User-Item matrix construction
The first thing we have to do is build the user-item matrix:

- Choose a similarity metric to calculate the similarity between items. Common metrics include cosine similarity, Pearson correlation coefficient, and Jaccard similarity.
- Calculate the similarity between each pair of items based on the ratings provided by users. This will result in an item-item similarity matrix.

#### Collaborative Filtering with Cosine Similarity

This code snippet demonstrates how to perform item-based collaborative filtering using cosine similarity. Collaborative filtering is a technique commonly used in recommendation systems to predict a user's preferences for items based on the preferences of similar users/items.

##### Libraries Used
- **pandas**: A powerful data manipulation library in Python.
- **numpy**: A library for numerical computing in Python.
- **sklearn.metrics.pairwise.cosine_similarity**: A function from scikit-learn used to compute the cosine similarity between vectors.

In [90]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame with the specified format
# Convert userId column to integer type if needed
if df['userId'].dtype != int:
    df['userId'] = df['userId'].astype(int)

# Get all unique user IDs and movie IDs
all_user_ids = np.unique(df['userId'])
all_movie_ids = np.unique(df['movieId'])

# Create a DataFrame with all combinations of user IDs and movie IDs
all_user_movie_pairs = np.array(np.meshgrid(all_user_ids, all_movie_ids)).T.reshape(-1, 2)
df_pairs = pd.DataFrame(all_user_movie_pairs, columns=['userId', 'movieId'])

# Merge the original DataFrame with all_user_movie_pairs to fill missing ratings with 0
df_filled = pd.merge(df_pairs, df, on=['userId', 'movieId'], how='left').fillna(0)

# Create user-item matrix using pivot_table
item_user_matrix = pd.pivot_table(df_filled, values='rating', index='userId', columns='movieId', fill_value=0)

# Convert user-item matrix to NumPy array for faster computation
item_user_array = item_user_matrix.to_numpy()


In [91]:
item_user_matrix.head()

movieId,1,4,15,30,43,89,104,108,122,146,...,174479,174551,175475,176371,176389,177593,179813,181413,185029,186587
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
item_user_array

array([[4. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 0. , 0. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

### Train-test split

The train-val-test split is a technique used in machine learning to evaluate the performance of a model. It involves dividing the dataset into three subsets: the training set, the validation set, and the test set.

The training set is used to train the model and optimize its parameters.
The validation set is used to fine-tune the model and select the best hyperparameters.
The test set is used to evaluate the final performance of the model on unseen data.
By using a train-val-test split, we can assess the model's performance on unseen data and ensure that it generalizes well to new examples. It helps prevent overfitting and provides a more reliable estimate of the model's performance in real-world scenarios.

In [93]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets
train_val, test = train_test_split(item_user_array, test_size=0.2, random_state=42)

# Split the training set into training and validation sets
train, val = train_test_split(train_val, test_size=0.2, random_state=42)

# Print the shapes of the datasets
print("Training set shape:", train.shape)
print("Validation set shape:", val.shape)
print("Test set shape:", test.shape)

# We are also going to do the split for the matrix df
# Split the user-item matrix into training and test sets
train_val_matrix, test_matrix = train_test_split(item_user_matrix, test_size=0.2, random_state=42)

# Split the training set matrix into training and validation sets
train_matrix, val_matrix = train_test_split(train_val_matrix, test_size=0.2, random_state=42)

# We print a ' ' to give some space inbetween lines
print(' ')

# Print the shapes of the matrix datasets
print("Training set matrix shape:", train_matrix.shape)
print("Validation set matrix shape:", val_matrix.shape)
print("Test set matrix shape:", test_matrix.shape)


Training set shape: (340, 487)
Validation set shape: (85, 487)
Test set shape: (107, 487)
 
Training set matrix shape: (340, 487)
Validation set matrix shape: (85, 487)
Test set matrix shape: (107, 487)


In [94]:
train_matrix

movieId,1,4,15,30,43,89,104,108,122,146,...,174479,174551,175475,176371,176389,177593,179813,181413,185029,186587
userId,,,,,,,,,,,,,,,,,,,,,
428,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
train

array([[0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [96]:
# Calculate cosine similarity between items using NumPy functions
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

# Calculate item-item similarity matrix for train data
item_similarity_matrix_train = np.zeros((train.shape[1], train.shape[1]))

for i in range(train.shape[1]):
    for j in range(i, train.shape[1]):
        item_similarity_matrix_train[i, j] = cosine_similarity(train[:, i], train[:, j])
        item_similarity_matrix_train[j, i] = item_similarity_matrix_train[i, j]

# Create a mapping from movie IDs to indices
movie_id_to_index = {movie_id: i for i, movie_id in enumerate(item_user_matrix.index)}
index_to_movie_id = {i: movie_id for movie_id, i in movie_id_to_index.items()}

# Create a mapping from user IDs to indices
user_id_to_index = {user_id: i for i, user_id in enumerate(item_user_matrix.columns)}
index_to_user_id = {i: user_id for user_id, i in user_id_to_index.items()}


C:\Users\Jaume\AppData\Local\Temp\ipykernel_4988\3392059385.py:6: RuntimeWarning: invalid value encountered in scalar divide
  similarity = dot_product / (norm_a * norm_b)


In [97]:
print(index_to_user_id)

{0: 1, 1: 4, 2: 15, 3: 30, 4: 43, 5: 89, 6: 104, 7: 108, 8: 122, 9: 146, 10: 290, 11: 303, 12: 305, 13: 346, 14: 353, 15: 358, 16: 363, 17: 389, 18: 393, 19: 416, 20: 423, 21: 478, 22: 504, 23: 522, 24: 551, 25: 556, 26: 605, 27: 627, 28: 640, 29: 694, 30: 708, 31: 728, 32: 790, 33: 810, 34: 835, 35: 901, 36: 953, 37: 1003, 38: 1006, 39: 1037, 40: 1040, 41: 1046, 42: 1207, 43: 1261, 44: 1266, 45: 1271, 46: 1272, 47: 1358, 48: 1373, 49: 1388, 50: 1391, 51: 1398, 52: 1415, 53: 1432, 54: 1446, 55: 1447, 56: 1507, 57: 1518, 58: 1564, 59: 1572, 60: 1580, 61: 1597, 62: 1625, 63: 1665, 64: 1757, 65: 1769, 66: 1816, 67: 1882, 68: 1940, 69: 1955, 70: 1964, 71: 1969, 72: 1995, 73: 2004, 74: 2013, 75: 2037, 76: 2046, 77: 2071, 78: 2107, 79: 2114, 80: 2148, 81: 2160, 82: 2171, 83: 2177, 84: 2273, 85: 2302, 86: 2310, 87: 2320, 88: 2324, 89: 2325, 90: 2336, 91: 2344, 92: 2356, 93: 2358, 94: 2366, 95: 2378, 96: 2381, 97: 2521, 98: 2540, 99: 2590, 100: 2606, 101: 2672, 102: 2746, 103: 2757, 104: 2803,

In [98]:
print(user_id_to_index)

{1: 0, 4: 1, 15: 2, 30: 3, 43: 4, 89: 5, 104: 6, 108: 7, 122: 8, 146: 9, 290: 10, 303: 11, 305: 12, 346: 13, 353: 14, 358: 15, 363: 16, 389: 17, 393: 18, 416: 19, 423: 20, 478: 21, 504: 22, 522: 23, 551: 24, 556: 25, 605: 26, 627: 27, 640: 28, 694: 29, 708: 30, 728: 31, 790: 32, 810: 33, 835: 34, 901: 35, 953: 36, 1003: 37, 1006: 38, 1037: 39, 1040: 40, 1046: 41, 1207: 42, 1261: 43, 1266: 44, 1271: 45, 1272: 46, 1358: 47, 1373: 48, 1388: 49, 1391: 50, 1398: 51, 1415: 52, 1432: 53, 1446: 54, 1447: 55, 1507: 56, 1518: 57, 1564: 58, 1572: 59, 1580: 60, 1597: 61, 1625: 62, 1665: 63, 1757: 64, 1769: 65, 1816: 66, 1882: 67, 1940: 68, 1955: 69, 1964: 70, 1969: 71, 1995: 72, 2004: 73, 2013: 74, 2037: 75, 2046: 76, 2071: 77, 2107: 78, 2114: 79, 2148: 80, 2160: 81, 2171: 82, 2177: 83, 2273: 84, 2302: 85, 2310: 86, 2320: 87, 2324: 88, 2325: 89, 2336: 90, 2344: 91, 2356: 92, 2358: 93, 2366: 94, 2378: 95, 2381: 96, 2521: 97, 2540: 98, 2590: 99, 2606: 100, 2672: 101, 2746: 102, 2757: 103, 2803: 104,

In [99]:
item_user_array.shape

(532, 487)

In [100]:
item_user_matrix.shape

(532, 487)

In [101]:
# We print the item_similarity_matrix
item_similarity_matrix_train

array([[1.        , 0.0143568 , 0.04177427, ...,        nan,        nan,
               nan],
       [0.0143568 , 1.        , 0.26332321, ...,        nan,        nan,
               nan],
       [0.04177427, 0.26332321, 1.        , ...,        nan,        nan,
               nan],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])

In [102]:
item_similarity_matrix_train.shape

(487, 487)

### Step 2: Neighborhood Selection
- Determine the neighborhood size, i.e., the number of most similar items to consider when predicting ratings for a target item.
- Select the most similar items for each item in your dataset based on their calculated similarities. This forms the neighborhood for each item.

#### Item-Based Neighborhoods and Ratings Aggregation

This code following snippet enhances the previous item-based collaborative filtering approach by considering ratings aggregation within the item neighborhoods.

##### Steps:

1. **Defining Neighborhood Size**:
   - The variable `neighborhood_size` determines the number of most similar items to consider in the neighborhood.

2. **Initializing Data Structure**:
   - An empty dictionary `item_neighborhoods` is initialized to store the neighborhoods for each item.

3. **Iterating Over Items**:
   - For each movie in the dataset:
     - All ratings for the current movie are extracted from the DataFrame (`df`).
     - Ratings aggregation is performed. In this example, the average rating for the movie is computed, but other aggregation methods can be used.
     - The similarity scores for the current movie are retrieved from the precomputed `item_similarity_matrix`.
     - Similarity scores are sorted in descending order, and the indices of the most similar items (excluding itself) are obtained.
     - These indices are converted back to movie IDs, forming the neighborhood for the current item.
     - The neighborhood for the current item is stored in the `item_neighborhoods` dictionary.

4. **Output**:
   - `item_neighborhoods`: A dictionary where keys are movie IDs, and values are lists of movie IDs representing the neighborhood of each item. Each movie's neighborhood includes movies with similar ratings and content.

##### Note:
- This approach considers both similarity in ratings and content (as captured by cosine similarity) when building item neighborhoods.
- Aggregating ratings within item neighborhoods helps in providing more personalized recommendations.
- The choice of rating aggregation method (e.g., mean, median) can impact the quality of recommendations and may need to be adjusted based on the characteristics of the dataset and user preferences.


In [103]:
# Step 2: Neighborhood Selection

# Define the neighborhood size
neighborhood_size = 5

# Initialize an empty dictionary to store item neighborhoods
item_neighborhoods = {}

# Iterate over each item (movie) index in the dataset
for movie_index in range(item_user_array.shape[1]):
    # Convert the item index to movie ID
    movie_id = index_to_movie_id[movie_index]
    
    # Extract all ratings for the current movie
    movie_ratings = item_user_array[:, movie_index]  # Changed to item_user_array
    
    # Aggregate ratings (e.g., compute the mean rating)
    movie_avg_rating = np.mean(movie_ratings)
    
    # Retrieve similarity scores for the current movie
    similarity_scores = item_similarity_matrix_train[movie_index]
    
    # Sort similarity scores in descending order and get indices of most similar items
    most_similar_indices = np.argsort(similarity_scores)[::-1][1:neighborhood_size+1]
    
    # Convert indices back to movie IDs to form the neighborhood
    neighborhood = [index_to_movie_id[idx] for idx in most_similar_indices]
    
    # Store the neighborhood for the current item in the item_neighborhoods dictionary
    item_neighborhoods[movie_id] = neighborhood

# Output: item_neighborhoods dictionary containing neighborhoods for each item
print(item_neighborhoods)


{1: [357, 465, 344, 467, 342], 2: [317, 451, 183, 453, 182], 3: [422, 417, 149, 412, 409], 4: [248, 162, 164, 436, 167], 5: [441, 448, 223, 224, 445], 6: [164, 417, 412, 409, 403], 7: [357, 344, 465, 342, 467], 9: [477, 444, 182, 441, 183], 10: [149, 418, 417, 159, 412], 11: [254, 453, 228, 451, 248], 12: [436, 441, 344, 342, 444], 13: [482, 310, 441, 293, 444], 14: [412, 324, 453, 323, 455], 15: [422, 417, 412, 409, 403], 16: [286, 451, 323, 453, 318], 17: [436, 159, 160, 162, 164], 18: [477, 444, 182, 441, 183], 19: [412, 89, 403, 91, 391], 20: [436, 448, 445, 444, 223], 21: [558, 417, 412, 409, 403], 22: [477, 160, 162, 164, 422], 23: [214, 182, 183, 184, 185], 24: [286, 418, 417, 412, 149], 25: [436, 448, 445, 444, 217], 27: [317, 459, 332, 330, 328], 28: [196, 441, 182, 183, 436], 29: [422, 441, 436, 432, 182], 30: [467, 324, 453, 323, 455], 31: [293, 455, 456, 327, 325], 32: [441, 449, 448, 217, 445], 33: [476, 342, 332, 330, 465], 34: [432, 445, 444, 196, 441], 36: [254, 453, 22

### Step 4: Rating Prediction

- For each target item and user pair where the user hasn't rated the target item:
Identify the neighborhood of similar items to the target item.
- Predict the rating for the target item using a weighted average of the ratings of the items in its neighborhood, where the weights are the similarities between the items and the target item.
- Adjust the prediction based on the user's average rating or other normalization techniques, if necessary.

In [104]:
# Train your model and create the item_neighborhoods dictionary using the training data (steps 1-3)

# Initialize an empty DataFrame to store predicted ratings
predicted_ratings = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in train_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Skip if the rating is non-zero (indicating a rating given by the user)
        if rating != 0:
            continue
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods:
            neighborhood = item_neighborhoods[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies for the user in the training set
                predicted_rating = user_ratings.mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)

# Display the head of the predicted_ratings DataFrame
predicted_ratings.head()


C:\Users\Jaume\AppData\Local\Temp\ipykernel_4988\1523731210.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)


movieId,1,4,15,30,43,89,104,108,122,146,...,174479,174551,175475,176371,176389,177593,179813,181413,185029,186587
userId,,,,,,,,,,,,,,,,,,,,,
428,0.063655,0.063655,0.063655,0.063655,0.063655,0.063655,0.056225,0.056225,0.063655,0.063655,...,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225
517,0.056225,0.084189,0.084189,0.084189,0.084189,0.084189,0.084189,0.056225,0.084189,0.084189,...,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225
197,0.006160,0.006160,0.006160,0.006160,0.006160,0.006160,0.006160,0.056225,0.006160,0.006160,...,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225
160,0.056225,0.131417,0.131417,0.131417,0.131417,0.131417,0.131417,0.056225,0.131417,0.131417,...,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225
67,0.025667,0.025667,0.025667,0.025667,0.025667,0.025667,0.025667,0.056225,0.025667,0.025667,...,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225,0.056225


In [105]:
predicted_ratings.shape

# We transform predicted_ratings into a df to export it as a csv
predicted_ratings_df = pd.DataFrame(predicted_ratings)

# We export the predicted_ratings_df as a csv
predicted_ratings_df.to_csv('predicted_ratings.csv')


(340, 487)

In [132]:
# We print the top 50 highest rated movies for user 517
predicted_ratings.loc[517].sort_values(ascending=False).head(50)

movieId
416      0.084189
346      0.084189
556      0.084189
551      0.084189
522      0.084189
504      0.084189
478      0.084189
423      0.084189
4        0.084189
393      0.084189
389      0.084189
363      0.084189
353      0.084189
358      0.084189
305      0.084189
303      0.084189
290      0.084189
146      0.084189
122      0.084189
104      0.084189
89       0.084189
43       0.084189
30       0.084189
15       0.084189
62344    0.056225
57526    0.056225
59667    0.056225
59738    0.056225
60904    0.056225
61073    0.056225
1        0.056225
62970    0.056225
63479    0.056225
63853    0.056225
64614    0.056225
55280    0.056225
65601    0.056225
55492    0.056225
50274    0.056225
55241    0.056225
55069    0.056225
54934    0.056225
54796    0.056225
54274    0.056225
53123    0.056225
52435    0.056225
50685    0.056225
50613    0.056225
50610    0.056225
66090    0.056225
Name: 517, dtype: float64

In [106]:
# We count the NaN values in the predicted_ratings dataframe
predicted_ratings.isnull().sum().sum()

0

In [107]:
def get_top_recommendations(user_id):
    """
    Get top movie recommendations for a given user using predicted ratings.

    Parameters:
        user_id (int): ID of the user for whom recommendations are to be generated.

    Returns:
        list: Top movie titles recommended for the user.
    """

    # Filter the DataFrame to get ratings for the specified user
    user_ratings = df[df['userId'] == user_id]
    
    # Check if the user has any ratings
    if user_ratings.empty:
        # If the user hasn't reviewed any movies, return the top 5 best-rated movies
        top_movies = df.groupby('title')['rating'].mean().sort_values(ascending=False).head(5).index.tolist()
        return top_movies
    
    # Sort the movies by rating in descending order
    sorted_movies = user_ratings.sort_values(by='rating', ascending=False)
    
    # Get the top 5 movie titles
    top_movie_titles = sorted_movies.head(5)['title'].tolist()
    
    # If the user has rated fewer than 5 movies, fill recommendations with the best-rated movies
    if len(top_movie_titles) < 5:
        # Calculate the number of additional recommendations needed
        additional_recommendations = 5 - len(top_movie_titles)
        
        # Get the best-rated movies that the user hasn't rated
        unrated_movies = df[~df['title'].isin(top_movie_titles)]
        best_unrated_movies = unrated_movies.groupby('title')['rating'].mean().sort_values(ascending=False).head(additional_recommendations).index.tolist()
        
        # Add the additional recommendations to the list
        top_movie_titles.extend(best_unrated_movies)
    
    return top_movie_titles[:5]  # Return only the top 5 recommendations


In [108]:
top_recommendations = []

# Iterate over each row (user) in the train_matrix
for user_index in range(train_matrix.shape[0]):
    # Get the actual user ID corresponding to the user index
    user_id = index_to_user_id[user_index]
    recommendations = get_top_recommendations(user_id)
    top_recommendations.append(recommendations)

# Output: List of top movie recommendations for each user in the train matrix
top_recommendations

# print size of top_recommendations
len(top_recommendations)


[['Robin Hood ',
  'Game, The ',
  'Messenger: The Story of Joan of Arc, The ',
  'King Kong ',
  'Transformers: The Movie '],
 ['Beautiful Thing ',
  'Unforgiven ',
  'Truth About Cats & Dogs, The ',
  'Men in Black (a.k.a. MIB) ',
  'Elizabeth '],
 ['10 Cloverfield Lane ',
  'What Women Want ',
  'Oblivion ',
  'Gran Torino ',
  'Source Code '],
 ['Wreck-It Ralph ',
  'Duel in the Sun ',
  'Human ',
  'Tom Segura: Completely Normal ',
  'Obsession '],
 ['Toy Story ',
  'Kazaam ',
  'Duel in the Sun ',
  'Human ',
  'Tom Segura: Completely Normal '],
 ['My Life as McDull (Mak dau goo si) ',
  'The Amazing Screw-On Head ',
  'Ryuzo and the Seven Henchmen ',
  'Dead Meat ',
  'Captain Newman, M.D. '],
 ['Chicago ',
  'Shrek ',
  'Robin Hood ',
  'Spirited Away (Sen to Chihiro no kamikakushi) ',
  'Harry Potter and the Deathly Hallows: Part 1 '],
 ['Duel in the Sun ',
  'Human ',
  'Decalogue, The (Dekalog) ',
  'Tom Segura: Completely Normal ',
  'Crossing Delancey '],
 ['Children of Me

340

In [109]:
print("Last version")

Last version


### Step 5: Model Evaluation
- We evaluate the performance of your ItemKNN algorithm using appropriate evaluation metrics such as Root Mean Squared Error (RMSE), Mean Absolute Error (MAE), or others.
- Split your dataset into training and testing sets to assess the model's predictive accuracy on unseen data.

In [110]:
train.shape

predicted_ratings.shape

(340, 487)

(340, 487)

In [111]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_model(train_matrix, predicted_ratings):
    """
    Evaluate the model's performance on the training data.

    Parameters:
        train_matrix (numpy.ndarray): Item-user matrix from the training data.
        predicted_ratings (pandas.DataFrame): Predicted ratings DataFrame for the training data.

    Returns:
        float: Mean Absolute Error (MAE) on the training data.
        float: Root Mean Squared Error (RMSE) on the training data.
    """
    # Ensure train_matrix and predicted_ratings have the same shape
    assert train_matrix.shape == predicted_ratings.shape, "Shapes of train_matrix and predicted_ratings are not consistent."

    # Convert predicted_ratings DataFrame to numpy array
    predicted_ratings_array = predicted_ratings.to_numpy()

    # Flatten the arrays
    train_ratings = train_matrix.flatten()
    predicted_ratings = predicted_ratings_array.flatten()

    # Filter out NaN values
    valid_indices = ~np.isnan(train_ratings) & ~np.isnan(predicted_ratings)
    train_ratings = train_ratings[valid_indices]
    predicted_ratings = predicted_ratings[valid_indices]

    # Calculate evaluation metrics
    mae = mean_absolute_error(train_ratings, predicted_ratings)
    rmse = np.sqrt(mean_squared_error(train_ratings, predicted_ratings))

    return mae, rmse

# Assuming train is the numpy array representing the item-user matrix
# Assuming predicted_ratings is the DataFrame representing the predicted ratings for the training data

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.11541909354372394
Root Mean Squared Error (RMSE) on Training Data: 0.4785515075368112


### Step 6: Parameter Tuning
- Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of your ItemKNN algorithm.
- Use cross-validation or other techniques to tune these parameters and avoid overfitting.

1. **Import Necessary Libraries:**
   - We import the required libraries for performing grid search cross-validation (`GridSearchCV`), creating custom scorers (`make_scorer`), and utilizing the `NearestNeighbors` algorithm.

2. **Define Cosine Similarity Function:**
   - We define a custom function `cosine_similarity` to compute the cosine similarity between two vectors. This function calculates the dot product of the vectors and divides it by the product of their norms.

3. **Define Custom Scorer:**
   - We create a custom scorer `cosine_similarity_scorer` using `make_scorer`, which enables us to use cosine similarity as the scoring metric during grid search cross-validation.

4. **Define Parameter Grid:**
   - We specify a parameter grid `param_grid` containing the hyperparameters to be tuned. In this case, we're tuning the number of neighbors (`n_neighbors`) and the distance metric (`metric`) for the `NearestNeighbors` algorithm.

5. **Initialize NearestNeighbors Model:**
   - We initialize the `NearestNeighbors` model without specifying any hyperparameters.

6. **Create GridSearchCV Object:**
   - We create a `GridSearchCV` object named `grid_search` with the specified parameter grid, cross-validation strategy (5-fold cross-validation), and custom scoring metric (`cosine_similarity_scorer`).

7. **Fit the Data:**
   - We fit the `item_user_matrix` data to the `grid_search` object to perform hyperparameter tuning. `item_user_matrix` typically contains the item-item similarity matrix computed using collaborative filtering techniques.

8. **Get Best Hyperparameters:**
   - After fitting the data, we retrieve the best hyperparameters selected by the grid search using the `best_params_` attribute of the `grid_search` object.

9. **Print Best Parameters:**
   - Finally, we print the best hyperparameters obtained from the grid search.



`scikit-learn (sklearn)`:

- Scikit-learn is a popular machine learning library in Python that provides simple and efficient tools for data analysis and modeling.
- It includes various modules for tasks such as classification, regression, clustering, dimensionality reduction, and model selection.
- The GridSearchCV class from scikit-learn is used for hyperparameter tuning through grid search along with cross-validation.
- The make_scorer function allows you to create a custom scoring function for use with GridSearchCV.
- The NearestNeighbors class provides functionality for unsupervised nearest neighbors learning, which can be used for tasks such as finding k-nearest neighbors for a given data point.

In [112]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_val_matrix)  # train_val_matrix contains item-user matrix

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


This code sets up a GridSearchCV object to perform hyperparameter tuning using the validation set (train_val_matrix). It explores different combinations of hyperparameters specified in the param_grid, evaluates them using 5-fold cross-validation (cv=5), and uses the cosine_similarity scorer to optimize the model's performance based on cosine similarity. Finally, it prints the best hyperparameters found during the search.

We have to perform the calculation of the predicted ratings for the train-val set to generate predicted ratings in order to evaluate the performance of our trained model on a validation dataset. 

If we computed the similarity matrix again specifically for the train_val set, it would essentially mean that we are using a different set of similarity measures for predicting ratings compared to what we used during training. This approach could lead to inconsistencies and potentially degrade the performance of our model. Here's what could happen:

1. **Inconsistency**: The similarity measures computed for the train_val set might differ from those computed for the training set due to variations in the data. As a result, the predicted ratings based on these new similarity measures may not align well with the predictions made during training, leading to inconsistency in the model's behavior.

2. **Overfitting**: Computing a new similarity matrix specifically for the train_val set might lead to overfitting on the validation data. The model may capture noise or idiosyncrasies present in the train_val set, which may not generalize well to unseen data.

3. **Increased Complexity**: Computing the similarity matrix again for the train_val set adds computational complexity and redundancy, especially if the similarity computation process is resource-intensive. This can result in longer training times and increased resource utilization.

Overall, it's generally recommended to use the same similarity measures or neighborhood definitions for both training and validation sets to ensure consistency and generalizability of the model.

In [113]:
# Assuming you have already trained your model and obtained the item_neighborhoods dictionary

# Initialize an empty DataFrame to store predicted ratings for train_val
predicted_ratings_train_val = pd.DataFrame(index=train_val_matrix.index, columns=train_val_matrix.columns)

# Iterate over each user and their ratings in the train_val set
for user_id, user_ratings in train_val_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        # Skip if the rating is non-zero (indicating a rating given by the user)
        if rating != 0:
            continue
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods:
            neighborhood = item_neighborhoods[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the train set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies for the user in the train set
                predicted_rating = user_ratings.mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_train_val.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_train_val.fillna(predicted_ratings_train_val.mean().mean(), inplace=True)


C:\Users\Jaume\AppData\Local\Temp\ipykernel_4988\568664895.py:33: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings_train_val.fillna(predicted_ratings_train_val.mean().mean(), inplace=True)


In [114]:
predicted_ratings_train_val.head()

predicted_ratings_train_val.shape

movieId,1,4,15,30,43,89,104,108,122,146,...,174479,174551,175475,176371,176389,177593,179813,181413,185029,186587
userId,,,,,,,,,,,,,,,,,,,,,
344,0.019507,0.019507,0.019507,0.019507,0.019507,0.019507,0.019507,0.054999,0.019507,0.019507,...,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999
465,0.041068,0.041068,0.041068,0.041068,0.041068,0.041068,0.041068,0.054999,0.041068,0.041068,...,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999
515,0.010267,0.010267,0.010267,0.010267,0.010267,0.010267,0.010267,0.054999,0.010267,0.010267,...,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999
357,0.054999,0.185832,0.185832,0.185832,0.185832,0.185832,0.185832,0.054999,0.185832,0.185832,...,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999
164,0.010267,0.010267,0.010267,0.010267,0.010267,0.010267,0.010267,0.054999,0.010267,0.010267,...,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999,0.054999


(425, 487)

Now we evaluate the model with the train-val item-user matrix.

In [115]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate item-item similarity matrix for train data
item_similarity_matrix_train = np.zeros((train.shape[1], train.shape[1]))

for i in range(train.shape[1]):
    for j in range(i, train.shape[1]):
        item_similarity_matrix_train[i, j] = cosine_similarity(train[:, i], train[:, j])
        item_similarity_matrix_train[j, i] = item_similarity_matrix_train[i, j]

# Assuming train_val_matrix and predicted_ratings_train_val are properly defined

# Now, you can evaluate the model using the evaluate_model function
mae, rmse = evaluate_model(train_val_matrix.to_numpy(), predicted_ratings_train_val)

print("Mean Absolute Error (MAE) on Train-Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Train-Validation Data:", rmse)


C:\Users\Jaume\AppData\Local\Temp\ipykernel_4988\3392059385.py:6: RuntimeWarning: invalid value encountered in scalar divide
  similarity = dot_product / (norm_a * norm_b)


Mean Absolute Error (MAE) on Train-Validation Data: 0.11270438375369393
Root Mean Squared Error (RMSE) on Train-Validation Data: 0.4715288573983273


### Step 7: Deployment

Once we have come up with the best parameters possible and trained the model with the whole train_validation set, we will test it on the test set.

In [116]:
# Assuming you have already trained your model and obtained the item_neighborhoods dictionary

# Now, we can evaluate the model's predicted ratings on the test set

# Initialize an empty DataFrame to store predicted ratings for the test set
predicted_ratings_test = pd.DataFrame(index=test_matrix.index, columns=test_matrix.columns)

# Iterate over each user and their ratings in the test set
for user_id, user_ratings in test_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        # Skip if the rating is non-zero (indicating a rating given by the user)
        if rating != 0:
            continue
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods:
            neighborhood = item_neighborhoods[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the train set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies for the user in the train set
                predicted_rating = user_ratings.mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_test.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)

# Now, you can evaluate the model's predicted ratings on the test set
mae, rmse = evaluate_model(test_matrix.to_numpy(), predicted_ratings_test)

print("Mean Absolute Error (MAE) on Test Data using Predicted Ratings from Train-Validation Set:", mae)
print("Root Mean Squared Error (RMSE) on Test Data using Predicted Ratings from Train-Validation Set:", rmse)


Mean Absolute Error (MAE) on Test Data using Predicted Ratings from Train-Validation Set: 0.1279818499213286
Root Mean Squared Error (RMSE) on Test Data using Predicted Ratings from Train-Validation Set: 0.4995626957908083


C:\Users\Jaume\AppData\Local\Temp\ipykernel_4988\438452714.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)


In [117]:
test_matrix

movieId,1,4,15,30,43,89,104,108,122,146,...,174479,174551,175475,176371,176389,177593,179813,181413,185029,186587
userId,,,,,,,,,,,,,,,,,,,,,
7,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
561,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
559,5.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,1.5,0.0
350,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
610,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
top_recommendations_test = []

# Iterate over each row (user) in the test_matrix
for user_index in range(test_matrix.shape[0]):
    # Get the actual user ID corresponding to the user index
    user_id = index_to_user_id[user_index]
    recommendations = get_top_recommendations(user_id)
    top_recommendations_test.append(recommendations)

# Output: List of top movie recommendations for each user in the test matrix
top_recommendations_test

# print size of top_recommendations
print("Size of top_recommendations_test:", len(top_recommendations_test))



[['Robin Hood ',
  'Game, The ',
  'Messenger: The Story of Joan of Arc, The ',
  'King Kong ',
  'Transformers: The Movie '],
 ['Beautiful Thing ',
  'Unforgiven ',
  'Truth About Cats & Dogs, The ',
  'Men in Black (a.k.a. MIB) ',
  'Elizabeth '],
 ['10 Cloverfield Lane ',
  'What Women Want ',
  'Oblivion ',
  'Gran Torino ',
  'Source Code '],
 ['Wreck-It Ralph ',
  'Duel in the Sun ',
  'Human ',
  'Tom Segura: Completely Normal ',
  'Obsession '],
 ['Toy Story ',
  'Kazaam ',
  'Duel in the Sun ',
  'Human ',
  'Tom Segura: Completely Normal '],
 ['My Life as McDull (Mak dau goo si) ',
  'The Amazing Screw-On Head ',
  'Ryuzo and the Seven Henchmen ',
  'Dead Meat ',
  'Captain Newman, M.D. '],
 ['Chicago ',
  'Shrek ',
  'Robin Hood ',
  'Spirited Away (Sen to Chihiro no kamikakushi) ',
  'Harry Potter and the Deathly Hallows: Part 1 '],
 ['Duel in the Sun ',
  'Human ',
  'Decalogue, The (Dekalog) ',
  'Tom Segura: Completely Normal ',
  'Crossing Delancey '],
 ['Children of Me

Size of top_recommendations_test: 107


# Item-based Classification (ItemKNN)

### Step 1: Data preparation
- Ensure you have a dataset with user-item interactions. Each interaction should include the user ID, item ID, and the corresponding label or class for classification.
- Preprocess the data as needed, including handling missing values, encoding categorical variables, and splitting into training and testing sets.

For this step we are going to resuse the `item_user_matrix` that we created in the first model built. 

### Step 2: Compute Item Similarity


In [119]:
import numpy as np

def item_neighborhood_selection(similarity_matrix, k=None, threshold=None):
    """
    Select a subset of similar items for each item based on similarity matrix.

    Parameters:
        similarity_matrix (numpy.ndarray): Item-item similarity matrix.
        k (int): Number of similar items to select (optional).
        threshold (float): Similarity threshold for selecting similar items (optional).

    Returns:
        dict: Dictionary containing similar items for each item.
    """
    num_items = similarity_matrix.shape[0]
    item_neighborhood = {}

    for i in range(num_items):
        if k is not None:
            # Select top-k similar items (excluding the item itself)
            similar_items_indices = np.argsort(similarity_matrix[i])[::-1][:k+1]
        elif threshold is not None:
            # Select items with similarity above threshold (excluding the item itself)
            similar_items_indices = np.where(similarity_matrix[i] > threshold)[0]

        # Remove the item itself from the neighborhood
        similar_items_indices = similar_items_indices[similar_items_indices != i]

        item_neighborhood[i] = similar_items_indices

    return item_neighborhood

# Example usage:
# Assuming you have an item-item similarity matrix 'item_similarity_matrix_train'
# and you want to select top-5 similar items for each item
k = 5
item_neighborhood = item_neighborhood_selection(item_similarity_matrix_train, k=k)

item_neighborhoods


{1: [357, 465, 344, 467, 342],
 2: [317, 451, 183, 453, 182],
 3: [422, 417, 149, 412, 409],
 4: [248, 162, 164, 436, 167],
 5: [441, 448, 223, 224, 445],
 6: [164, 417, 412, 409, 403],
 7: [357, 344, 465, 342, 467],
 9: [477, 444, 182, 441, 183],
 10: [149, 418, 417, 159, 412],
 11: [254, 453, 228, 451, 248],
 12: [436, 441, 344, 342, 444],
 13: [482, 310, 441, 293, 444],
 14: [412, 324, 453, 323, 455],
 15: [422, 417, 412, 409, 403],
 16: [286, 451, 323, 453, 318],
 17: [436, 159, 160, 162, 164],
 18: [477, 444, 182, 441, 183],
 19: [412, 89, 403, 91, 391],
 20: [436, 448, 445, 444, 223],
 21: [558, 417, 412, 409, 403],
 22: [477, 160, 162, 164, 422],
 23: [214, 182, 183, 184, 185],
 24: [286, 418, 417, 412, 149],
 25: [436, 448, 445, 444, 217],
 27: [317, 459, 332, 330, 328],
 28: [196, 441, 182, 183, 436],
 29: [422, 441, 436, 432, 182],
 30: [467, 324, 453, 323, 455],
 31: [293, 455, 456, 327, 325],
 32: [441, 449, 448, 217, 445],
 33: [476, 342, 332, 330, 465],
 34: [432, 445, 44

In [120]:
item_similarity_matrix_train.shape[0]

487

### Step 3: Class Label Propagation

In [121]:
print(item_neighborhood)

{0: array([486, 311, 403, 298, 405, 296], dtype=int64), 1: array([486, 276, 391, 154, 393, 153], dtype=int64), 2: array([486, 365, 360, 126, 355, 352], dtype=int64), 3: array([486, 212, 138, 140, 377, 143], dtype=int64), 4: array([486, 381, 388, 190, 191, 385], dtype=int64), 5: array([486, 140, 360, 355, 352, 349], dtype=int64), 6: array([486, 311, 298, 403, 296, 405], dtype=int64), 7: array([486, 413, 384, 153, 381, 154], dtype=int64), 8: array([486, 126, 361, 360, 135, 355], dtype=int64), 9: array([486, 217, 393, 195, 391, 212], dtype=int64), 10: array([486, 377, 381, 298, 296, 384], dtype=int64), 11: array([486, 418, 269, 381, 255, 384], dtype=int64), 12: array([486, 355, 280, 393, 279, 395], dtype=int64), 13: array([486, 365, 360, 355, 352, 349], dtype=int64), 14: array([486, 248, 391, 279, 393, 277], dtype=int64), 15: array([486, 377, 135, 136, 138, 140], dtype=int64), 16: array([486, 413, 384, 153, 381, 154], dtype=int64), 17: array([486, 355,  75, 349,  77, 342], dtype=int64), 1

In [122]:
import pandas as pd

def rating_propagation(item_neighborhood, item_user_matrix):
    """
    Propagate ratings from similar items to the target item using averaging.

    Parameters:
        item_neighborhood (dict): Dictionary containing similar items for each item.
        item_user_matrix (pandas.DataFrame): User-item matrix containing user ratings.

    Returns:
        dict: Dictionary containing propagated ratings for each item.
    """
    propagated_ratings = {}

    for target_item_index, similar_item_indices in item_neighborhood.items():
        ratings = []
        for similar_item_index in similar_item_indices:
            if similar_item_index < item_user_matrix.shape[0]:
                rating = item_user_matrix.iloc[similar_item_index].mean()  # Mean rating for similar item
                ratings.append(rating)
            else:
                print(f"Index {similar_item_index} is out of bounds.")

        propagated_rating = sum(ratings) / len(ratings) if ratings else None
        propagated_ratings[target_item_index] = propagated_rating

    return propagated_ratings

# Example usage:
# Assuming you have item_neighborhood and train DataFrame with user-item interactions
# where each cell contains a user's rating for an item
propagated_ratings = rating_propagation(item_neighborhood, item_user_matrix)

print(propagated_ratings)


{0: 0.05475701574264202, 1: 0.11310746064339494, 2: 0.029603011635865845, 3: 0.03182751540041068, 4: 0.11447638603696099, 5: 0.023271731690622865, 6: 0.05475701574264202, 7: 0.11516084873374403, 8: 0.02498288843258042, 9: 0.054414784394250515, 10: 0.022073921971252564, 11: 0.025496235455167698, 12: 0.04021218343600274, 13: 0.028747433264887063, 14: 0.08247775496235456, 15: 0.041752224503764555, 16: 0.11516084873374403, 17: 0.08281998631074605, 18: 0.11242299794661192, 19: 0.024982888432580425, 20: 0.07905544147843942, 21: 0.08915126625598906, 22: 0.02446954140999315, 23: 0.14082819986310746, 24: 0.07067077344284736, 25: 0.08042436687200548, 26: 0.10130047912388775, 27: 0.04123887748117728, 28: 0.04449007529089665, 29: 0.13997262149212866, 30: 0.04911019849418206, 31: 0.037987679671457907, 32: 0.054414784394250515, 33: 0.04911019849418206, 34: 0.02292950034223135, 35: 0.1856605065023956, 36: 0.04911019849418207, 37: 0.08247775496235456, 38: 0.1266255989048597, 39: 0.05886379192334018, 4

### Step 4: Classify Items

- For each item in the testing set, find its nearest neighbors based on the precomputed item neighborhoods.
- Determine the class labels of these neighbors.
- Use a strategy such as majority voting to assign the class label to the item.

In [128]:
def perform_classification(item_neighborhood, propagated_ratings):
    """
    Perform classification by aggregating the propagated ratings from similar items.

    Parameters:
        item_neighborhood (dict): Dictionary containing similar items for each item.
        propagated_ratings (dict): Propagated ratings for each item.

    Returns:
        dict: Dictionary containing classified ratings for each item.
    """
    classified_ratings = {}

    for target_item_index, similar_item_indices in item_neighborhood.items():
        rating_sum = 0
        count = 0
        for similar_item_index in similar_item_indices:
            propagated_rating = propagated_ratings.get(similar_item_index)
            if propagated_rating is not None:
                rating_sum += propagated_rating
                count += 1
        
        if count > 0:
            classified_rating = rating_sum / count
            classified_ratings[target_item_index] = classified_rating
        else:
            # If there are no similar items or propagated ratings available, assign NaN
            classified_ratings[target_item_index] = float('nan')

    return classified_ratings

# Example usage:
classified_ratings = perform_classification(item_neighborhood, propagated_ratings)


In [130]:
# We check for the NaN values in the classified_ratings
pd.Series(classified_ratings).isnull().sum()

0

### Step 6: Recommendation Generation

Once the model is trained and evaluated, you can use it to generate item recommendations for users. For a given user, recommend items that are highly rated or predicted to be liked based on the model's predictions.

In [ ]:
def predict_user_ratings_and_recommendations(item_user_matrix, item_neighborhood, classified_ratings, top_n=5):
    """
    Predict ratings for users based on the majority class label in the item neighborhood.
    Also, generate item recommendations for each user.

    Parameters:
        item_user_matrix (pandas.DataFrame): User-item matrix containing ratings or interactions.
        item_neighborhood (dict): Dictionary containing similar items for each item.
        classified_ratings (dict): Classified ratings for each item.
        top_n (int): Number of top recommendations to generate (default is 5).

    Returns:
        tuple: Tuple containing predicted ratings for each user and recommended items.
    """
    user_ratings = {}
    user_recommendations = {}

    for user_id, user_interactions in item_user_matrix.iterrows():
        ratings = {}
        recommendations = []

        # Iterate over the items interacted by the user
        for item_id, interaction in user_interactions.items():
            # Check if the item has a classified rating and is in the item neighborhood
            if item_id in classified_ratings and item_id in item_neighborhood:
                # Get the classified rating for the item
                classified_rating = classified_ratings[item_id]
                # Add the classified rating to the user's ratings
                ratings[item_id] = classified_rating
                # Add the item to recommendations
                recommendations.extend(item_neighborhood[item_id][:top_n])

        # Determine the top recommended items for the user
        recommendations = list(set(recommendations))  # Remove duplicates
        recommendations = [item_id for item_id in recommendations if item_id not in user_interactions.index]  # Remove items already interacted by the user
        user_recommendations[user_id] = recommendations[:top_n]

        # Assign the predicted ratings to the user
        user_ratings[user_id] = ratings

    return user_ratings, user_recommendations

# Example usage:
predicted_user_ratings, user_recommendations = predict_user_ratings_and_recommendations(item_user_matrix, item_neighborhood, classified_ratings)
print("Predicted ratings for users:", predicted_user_ratings)
print("Recommended items for users:", user_recommendations)


Predicted ratings for users: {1: {1: 0.08352156057494868, 4: 0.09027492584987452, 15: 0.09027492584987452, 30: 0.09027492584987452, 43: 0.09027492584987452, 89: 0.09027492584987452, 104: 0.08102897558749715, 108: 0.09027492584987452, 122: 0.09027492584987452, 146: 0.09027492584987452, 290: 0.09027492584987452, 303: 0.09027492584987452, 305: 0.09027492584987452, 346: 0.09027492584987452, 353: 0.09027492584987452, 358: 0.09027492584987452, 363: 0.09027492584987452, 389: 0.06327287246178416, 393: 0.06327287246178416, 416: 0.06327287246178416, 423: 0.09027492584987452, 478: 0.06327287246178416}, 2: {1: 0.08352156057494868, 4: 0.09027492584987452, 15: 0.09027492584987452, 30: 0.09027492584987452, 43: 0.09027492584987452, 89: 0.09027492584987452, 104: 0.08102897558749715, 108: 0.09027492584987452, 122: 0.09027492584987452, 146: 0.09027492584987452, 290: 0.09027492584987452, 303: 0.09027492584987452, 305: 0.09027492584987452, 346: 0.09027492584987452, 353: 0.09027492584987452, 358: 0.09027492

### Step 5: Evaluate Model Performance

- Assess the performance of the ItemKNN classifier using evaluation metrics such as accuracy, precision, recall, F1-score, or area under the ROC curve (AUC).
- Compare the performance of different models with varying neighborhood sizes or similarity metrics to identify the optimal configuration.

In [131]:
import numpy as np

def evaluate_model_performance(train_matrix, classified_ratings):
    """
    Evaluate the performance of the model using Mean Absolute Error (MAE) and Root Mean Squared Error (RMSE) 
    on the train split.

    Parameters:
        train_matrix (numpy.ndarray): Item-user matrix from the training data.
        classified_ratings (dict): Dictionary containing classified ratings for each item.

    Returns:
        tuple: Tuple containing MAE and RMSE
    """
    # Convert train matrix DataFrame to NumPy array
    train_matrix_array = train_matrix.to_numpy()

    # Flatten the train matrix
    actual_ratings = train_matrix_array.flatten()

    # Filter out NaN values
    valid_indices = ~np.isnan(actual_ratings)
    actual_ratings = actual_ratings[valid_indices]

    # Convert classified ratings dictionary to numpy array
    predicted_ratings = np.array([classified_ratings.get(i, np.nan) for i in range(len(actual_ratings))])

    # Filter out NaN values in predicted ratings
    valid_predicted_indices = ~np.isnan(predicted_ratings)
    actual_ratings = actual_ratings[valid_predicted_indices]
    predicted_ratings = predicted_ratings[valid_predicted_indices]

    # Calculate Mean Absolute Error (MAE)
    mae = np.mean(np.abs(actual_ratings - predicted_ratings))

    # Calculate Root Mean Squared Error (RMSE)
    rmse = np.sqrt(np.mean((actual_ratings - predicted_ratings) ** 2))

    return mae, rmse

# Example usage:
# Assuming train_matrix and classified_ratings are properly defined
mae, rmse = evaluate_model_performance(train_matrix, classified_ratings)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)


Mean Absolute Error (MAE): 0.14244746479420914
Root Mean Squared Error (RMSE): 0.43205576049998556


### Step 7: Parameter Tuning

Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of your ItemKNN algorithm.
Use techniques like cross-validation to tune these parameters and avoid overfitting.

In [127]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Define a function to compute cosine similarity
def cosine_similarity(X, Y):
    """
    This function computes the cosine similarity between two vectors X and Y.

    Parameters:
        X (ndarray): First vector
        Y (ndarray): Second vector

    Returns:
        float: Cosine similarity between X and Y
    """
    return np.dot(X, Y) / (np.linalg.norm(X) * np.linalg.norm(Y))

# Define a custom scorer based on cosine similarity
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model_classification = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model_classification, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(item_user_matrix)  # item_user_matrix contains item-item similarity matrix

# Get the best hyperparameters
best_params_classification = grid_search.best_params_

print("Best parameters:", best_params_classification)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


### Step 7: Deployment

- If you want to predict the class labels for new items, repeat steps 4 and 5 using the entire dataset (training + testing) to build the final model.
- Use the trained model to predict the class labels for new items based on their nearest neighbors.